In [1]:
pip install watchdog

In [3]:
import os
import shutil
import psutil
import time
from datetime import datetime

# Función para obtener la letra de unidades de almacenamiento conectadas
def get_usb_drives():
    drives = []
    partitions = psutil.disk_partitions()
    for partition in partitions:
        if 'removable' in partition.opts:
            drives.append(partition.mountpoint)
    return drives

# Función para copiar los archivos más recientes del último día a una carpeta en el escritorio
def copy_recent_files_from_usb(usb_directory):
    today = datetime.now().date()
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    destination_directory = os.path.expanduser(f"~/Desktop/Archivos_{current_time}")
    
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
    
    # Recorrer los archivos en la memoria USB
    for foldername, subfolders, filenames in os.walk(usb_directory):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            file_creation_time = datetime.fromtimestamp(os.path.getmtime(file_path)).date()
            
            # Copiar archivos creados o modificados hoy
            if file_creation_time == today:
                dest_path = os.path.join(destination_directory, filename)
                print(f"Copiando archivo: {file_path} a {dest_path}")
                shutil.copy2(file_path, dest_path)
    print(f"Archivos copiados en: {destination_directory}")

# Función para monitorear nuevas conexiones USB
def monitor_usb_connections():
    connected_drives = set(get_usb_drives())
    print("Esperando que se conecte una unidad USB...")

    while True:
        time.sleep(5)  # Revisa cada 5 segundos
        current_drives = set(get_usb_drives())

        # Detectar nuevas unidades conectadas
        new_drives = current_drives - connected_drives

        if new_drives:
            for drive in new_drives:
                print(f"Nueva unidad conectada: {drive}")
                copy_recent_files_from_usb(drive)

        connected_drives = current_drives

if __name__ == "__main__":
    monitor_usb_connections()


Esperando que se conecte una unidad USB...
Nueva unidad conectada: D:\
Copiando archivo: D:\hola.xlsx a C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-54-27\hola.xlsx
Copiando archivo: D:\hola.docx a C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-54-27\hola.docx
Archivos copiados en: C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-54-27
Nueva unidad conectada: D:\
Copiando archivo: D:\hola.xlsx a C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-57-13\hola.xlsx
Copiando archivo: D:\hola.docx a C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-57-13\hola.docx
Archivos copiados en: C:\Users\Camilo/Desktop/Archivos_2024-10-08_12-57-13


KeyboardInterrupt: 